### Ansible

```
Using Ansible base steps for the future data hub environment.

Install on mac:
brew update
brew install ansible
(or us pip to install ansible module using python alternatively)

test:
ansible --version

Use git as version system:
- use master as dev branch (to prevent accidental pushes to prod) 
- create prod branch (or test etc)
- USe branches for long term changes (feature branch)
- Always deploy right after pushing to production.
- Use small changes, not 1 big change.

```

### Testing Ansible without cloud provider

```
Well to prevent local gluther just create a server in aws manual
We create an ec nano server, with port 22 and 33434 (ping) open to the your public ip. Also we open port 80 for a webserver.
Download the key text put this in a <server>.pem file ($HOME/.ssh/<server>.pem)
Check out the public ip in the aws console: 

Go to the ec2 server
cd /Users/michelnossin/.ssh
chmod 400 *.pem
ssh -i michel-ansible-test.pem ec2-user@<public ip>
exit

#Now add the key to ssh so we dont need to specify
ssh-add michel-ansible-test.pem
ssh ec2-user@<public ip>

```



```
Some ansible tests:
ansible --help
<check out ansi configdir near inventory part!>
ansible all -i 35.157.62.157, -m ping -u ec2-user
ansible all -i 35.157.62.157, -m shell -a "echo 'hallo'" -u ec2-user

-m is for modules, with optional -a as arguments

```

### Playbooks

example, will put in current repository.
execute with: 
ansible-playbook -i 35.157.62.157, http-server.yaml

``` 
---
- hosts: all
  remote_user: ec2-user
  become: True
  tasks:
  - name: Ensure the HTTPd package is installed
    yum:
      name: httpd
      state: present
  - name: Ensure the HTTPd service is enabled and running
    service:
      name: httpd
      state: started
      enabled: True

```    
        
 Running multiple times gives changed values instead of ok, this means idempotency. Monitor change values therefor.
 debug: run with -v , or multiple like -vvv to get extra info

Let's add in same yml file set and get of variables, to be used while
reusing yaml playbooks.
```
variables:

tasks:
- name: Set variable 'x' 
  set_fact:
    x: 'michel'
- name: Get variable 'x'
  debug:
    msg: '{{ x }}'
``` 

Or pass -e 'x=michel' at the prompt, so set is not needed. Only issue, you lose that configuration, its not stored

Optionally add this to make sure our webserver can pass firewall. We will not add this as our webserver allready works.
```
- name: Ensure HTTP can pass the firewall
       firewalld:
         service: http
         state: enabled
         permanent: True
         immediate: True
       become: True
     - name: Ensure HTTPS can pass the firewall
       firewalld:
         service: https
         state: enabled
         permanent: True
         immediate: True
become: True
```

Lets publish our page:
```
- name: Ensure the website is present and updated
       template:
         src: index.html.j2
         dest: /var/www/html/index.html
         owner: root
         group: root
         mode: 0644

We will put this in tje j2 file

<html>
       <body>
           <h1>Hello Michel <p> born around {{ ansible_date_time.date }}</p></h1>
       </body>
</html>

```

go to http://35.157.62.157 , it works

Other usefull snippets:
```
- name: Ensure the MOTD file is present and updated
     template:
       src: motd
       dest: /etc/motd
       owner: root
       group: root
       mode: 0644
become: True

With text, will give all used settings during creation:
               This system is managed by Ansible
     Any change done on this system could be overwritten by Ansible
   OS: {{ ansible_distribution }} {{ ansible_distribution_version }}
   Hostname: {{ inventory_hostname }}
   eth0 address: {{ ansible_eth0.ipv4.address }}
   All connections are monitored and recorded
       Disconnect IMMEDIATELY if you are not an authorized user
       
Or latest package of everythin (but is dangerous?) is version
are idempotent.

       - name: Ensure we have last version of every package
     yum:
name: "*"
       state: latest
     become: True
    
Timezone, as we are in CET , set it to CET

- name: Ensure the timezone is set to CET
     file:
       src: /usr/share/zoneinfo/CET
       dest: /etc/localtime
       state: link
       force: yes
become: True

Change to hostname as mentioned in our config
- name: Ensure the hostname is the same of the inventory
     hostname:
       name: "{{ inventory_hostname }}"
     become: True
     

Finally , if you have a server with root. Run this to get a sudo user 
and disble root later.

---
   - hosts: all
     user: root
     tasks:
     - name: Ensure ansible user exists
       user:
         name: ansible
         state: present
         comment: Ansible
     - name: Ensure ansible user accepts the SSH key
       authorized_key:
         user: ansible
         key: https://github.com/fale.keys
      state: present
     - name: Ensure the ansible user is sudoer with no password required
       lineinfile:
         dest: /etc/sudoers
         state: present
         regexp: '^ansible ALL\='
         line: 'ansible ALL=(ALL) NOPASSWD:ALL'
         validate: 'visudo -cf %s'
    
```
Note exclusive option in authorized keys can be used to remove existing
keys.

### Jinja2 template

```
Use {{ variable}} to print, or {{ object.variable }} for properties
Use methodes like filters: {{ variable | capitalize }}
USe conditions like tests : 
            {% if ansible_eth0.active == True %}
                      <p>eth0 address {{ ansible_eth0.ipv4.address }}.</p>
           {% endif %}
Use cycles:
     {% for address in ansible_all_ipv4_addresses %}
                   <li>{{ address }}</li>
    {% endfor %}

```

### inventory file

ansible-playbook -i hosts http-server.yaml



```
put the ip in file called hosts in the current directory . If you ommit -i hosts it looks for location shown by ansible --help) 

Or use groups in the file: 
(my default location is /usr/local/etc/ansible/hosts default)

[webserver]
web1.nossy.com
web2.nossy.com
web[3:10].nossy.com

[dbserver]
db1.nossy.com domainname=somesource.test.com

[dbserver:vars]
https_enables=True
dil_great=True

Finally in the playbook , -hosts: all , or -hosts: webserver

As seen use regular exp like  db[1:2].nossy.com , using regular expression
in hostfiles make it way shorter

host variables
: For exaomple domainname shows which source domain is used.

group variables:
per group define a [group:vars] group, and set variables underneath.

variables files:
In case you have too many var: create a dir host_vars and create file 
named has host mentioned in hostfile. For groups make a group_vars folder,
and name the file like the group
e.g implementing previous example
file host_vars/db1.nossy.com with contents domainname=somesource.test.com
file group_vars/dbserver with contents https_enables=True and dil_great=True

Dynamic inventory scripts:
should be made in python. In the cloud host inventory files is not handy
as server are automatically crested and destroyed. rather you query the 
cload which generates the iventory dynamically . many dyn, scripts
already are made.

to start using ec2:
make sure ansible is installed
Make suree boto (python -m pip install boto) is done
Put in credentials:
# vi ∼/.boto
1 [Credentials]
2 aws_access_key_id=YOURACCESSKEY
3 aws_secret_access_key=YOURSECRETKEY
Create yml with localhost as hostname.
In yaml use the modules calling aws.

```

Example notebook ec2:
```
Always use localhost. and therefor just out in yaml itself is the best place. 
call using : ansible-playbook igw-info.yaml
call example on localhost: ansible-playbook localstuff.yml

- hosts: localhost
  connection: local
  tasks:
    - name: Gather facts about all Internet Gateways for an account or profile
      ec2_vpc_igw_facts:
        region: "{{ item }}"
      register: igw_facts
      with_items:
       - eu-central-1
       - eu-west-1
    - name: Get variable 'igw_facts'
      debug:
        msg: '{{ igw_facts }}'
        
standard iteration : with_items
Do same action with different variables.

With nested if you want to create cartesian product.
say you have set  vars: users: -alice -bob and folders: -mail -html

-name ensure folder exists
  <check if exists /home/{{ item.0 }}/{{ item.1 }}>
  with_nested:
  - "{{ users }}"
  - "{{ folders }}"
  
with_fileglobs can be used to iterate over loop of files
- name: copy rt* files to tmp
  copy:
     src: '{{ item }}'
     dst: '/tmp' 
  with_fileglob:
  - '/etc/rt*'

to use number range:
with_Sequence: start=1 end=10 , also use using '{{ item }}'

use local_action to execute a cmd on the host running ansible
example :
- name: Count processes running on the local system
           local_action: shell ps | wc -l
           register: local_processes_number
         - name: Print local running processes
           debug:
msg: '{{ local_processes_number.stdout }}'
You could also yse deletage_to: locahost (or other host)

Conditionals:
on a task add a check on a variable:
when: ansible_os_family == 'RedHat'
or when: backup (True or False boolean)
when: backup is not defined (does the opposite)
this can be used like eg:
- name: check if backup var is set
  fail:
    msg: "set back var"
  when: backup is not defined
Fail will stopt the script, the when: x is not defined is the check itself


```

Checks and errors:
also execute like : ansible-playbook localstuff.yaml 

```
- hosts: localhost
  connection: local
  vars:
    - backup: True
  tasks:
    - name: Count processes running on the local system
      local_action: shell ps | wc -l
      register: local_processes_number
    - name: Print local running processes
      debug:
        msg: '{{ local_processes_number.stdout }}'
    - name: Is backup set
      fail:
        msg: "Set backup env first!"
      when: backup is not defined
    - name: Is michel set
      local_action: shell echo "michel is set"
      when: michel is defined
 
 Try to create small yaml which can include eachothers.
 use, optionaly pass some variables to be filled in:
 - include: some.yaml var1="michel"
 
 handlers: only get execute after a notify (event) from another task.
 for example if you want to restart http server aftter somr config
 file and other task are done.
 
       copy:
         src: website.conf
         dest: /etc/httpd/conf.d
become: True
       notify: Restart HTTPd
     handlers:
     - name: Restart HTTPd
       service:
         name: httpd
         state: restarted
        become: True

 
 
 ```

Changing the local yaml file to add handlers
```
- hosts: localhost
  connection: local
  vars:
    - backup: True
  tasks:
    - name: Count processes running on the local system
      local_action: shell ps | wc -l
      register: local_processes_number
    - name: Print local running processes
      debug:
        msg: '{{ local_processes_number.stdout }}'
    - name: Is backup set
      fail:
        msg: "Set backup env first!"
      when: backup is not defined
    - name: Is michel is not set
      local_action: shell echo "michel is NOT set"
      when: michel is  not defined
      notify: Some handler
  handlers:
    - name: Some handler
      local_action: shell echo "I sm notified by missing michel"
```

### Roles

set of books, templates, files and variables for a goal. eg database tole, webserver role etc.

Create a projects like this:
```
<hosts>
<ansible.cfg>
master.yml
playbooks
  firstrun.yml
  groups
     database.yml
     webserver.yml
roles
  common
    tasks
       main.yml
    templates
       motd
  database
  webserver
  
next put common tasks in roles/common/tasks
Just the -name parts
  
  in database.yaml group we mention common role:
- hosts: database
  user: ansible
  roles:
  - common
  
  
```


                
                        
                 
                            
                             
            
                  
                    
                     